In [150]:
using Pkg
Pkg.add("GLM")
Pkg.add("StatsModels")

   Resolving package versions...
  No Changes to `C:\Users\sophi\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\sophi\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\sophi\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\sophi\.julia\environments\v1.10\Manifest.toml`


# 3. Bootstrapping and Decision Trees

3.1 Generar el dataset, transformar las variables categóricas en dummies.

In [7]:
println("Generamos el dataset base:")
using RDatasets, DataFrames
Hitters = dataset("ISLR", "Hitters")
Hitters[1:5, :]

Generamos el dataset base:


Row,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Cat…,Cat…,Int32,Int32,Int32,Float64?,Cat…
1,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,missing,A
2,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
3,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
4,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
5,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [10]:
Hitters

Row,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Cat…,Cat…,Int32,Int32,Int32,Float64?,Cat…
1,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,missing,A
2,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
3,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
4,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
5,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
6,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A
7,185,37,1,23,8,21,2,214,42,1,30,9,24,N,E,76,127,7,70.0,A
8,298,73,0,24,24,7,3,509,108,0,41,37,12,A,W,121,283,9,100.0,A
9,323,81,6,26,32,8,2,341,86,6,32,34,8,N,W,143,290,19,75.0,N


In [11]:
println("Identificamos los valores faltantes:")
describe(Hitters, :nmissing)

Identificamos los valores faltantes:


Row,variable,nmissing
,Symbol,Int64
1,AtBat,0
2,Hits,0
3,HmRun,0
4,Runs,0
5,RBI,0
6,Walks,0
7,Years,0
8,CAtBat,0
9,CHits,0


In [12]:
println("Habiendo identificado que solo hay valores faltantes en la columna 'Salario', nos deshacemos de estos")
Hitters = dropmissing(Hitters, :Salary);

Habiendo identificado que solo hay valores faltantes en la columna 'Salario', nos deshacemos de estos


In [ ]:
println("Definimos nuestro vector y la matriz objetivos")
using MLJ
Y, X = unpack(Hitters, ==(:Salary), !=(:Salary));

Definimos nuestro vector y la matriz objetivos


In [14]:
println("Verificamos los tipos de datos")
schema(X)

Verificamos los tipos de datos


┌───────────┬───────────────┬─────────────────────────────────┐
│ names     │ scitypes      │ types                           │
├───────────┼───────────────┼─────────────────────────────────┤
│ AtBat     │ Count         │ Int32                           │
│ Hits      │ Count         │ Int32                           │
│ HmRun     │ Count         │ Int32                           │
│ Runs      │ Count         │ Int32                           │
│ RBI       │ Count         │ Int32                           │
│ Walks     │ Count         │ Int32                           │
│ Years     │ Count         │ Int32                           │
│ CAtBat    │ Count         │ Int32                           │
│ CHits     │ Count         │ Int32                           │
│ CHmRun    │ Count         │ Int32                           │
│ CRuns     │ Count         │ Int32                           │
│ CRBI      │ Count         │ Int32                           │
│ CWalks    │ Count         │ Int32     

In [15]:
println("Tranformamos los datos 'Multiclass' en dummies continuas y los datos 'Count' en continuos")
Onehotencoder = @load OneHotEncoder pkg=MLJModels verbosity=0

ohe = Onehotencoder(features = [:League, :Division, :NewLeague])
ohe_machine = machine(ohe, X)
fit!(ohe_machine);
X = MLJ.transform(ohe_machine, X);
coerce!(X, Count => Continuous);

schema(X)

Tranformamos los datos 'Multiclass' en dummies continuas y los datos 'Count' en continuos


[ Info: Training machine(OneHotEncoder(features = [:League, :Division, :NewLeague], …), …).
[ Info: Spawning 2 sub-features to one-hot encode feature :League.
[ Info: Spawning 2 sub-features to one-hot encode feature :Division.
[ Info: Spawning 2 sub-features to one-hot encode feature :NewLeague.


┌──────────────┬────────────┬─────────┐
│ names        │ scitypes   │ types   │
├──────────────┼────────────┼─────────┤
│ AtBat        │ Continuous │ Float64 │
│ Hits         │ Continuous │ Float64 │
│ HmRun        │ Continuous │ Float64 │
│ Runs         │ Continuous │ Float64 │
│ RBI          │ Continuous │ Float64 │
│ Walks        │ Continuous │ Float64 │
│ Years        │ Continuous │ Float64 │
│ CAtBat       │ Continuous │ Float64 │
│ CHits        │ Continuous │ Float64 │
│ CHmRun       │ Continuous │ Float64 │
│ CRuns        │ Continuous │ Float64 │
│ CRBI         │ Continuous │ Float64 │
│ CWalks       │ Continuous │ Float64 │
│ League__A    │ Continuous │ Float64 │
│ League__N    │ Continuous │ Float64 │
│ Division__E  │ Continuous │ Float64 │
│ Division__W  │ Continuous │ Float64 │
│ PutOuts      │ Continuous │ Float64 │
│ Assists      │ Continuous │ Float64 │
│ Errors       │ Continuous │ Float64 │
│ NewLeague__A │ Continuous │ Float64 │
│ NewLeague__N │ Continuous │ Float64 │


3.2 Dividir el sample en dos sets: Training (90%) y Test (10%)

In [16]:
train, test = partition(eachindex(Y), 0.9, rng = 1);

In [115]:
Hitters2 = hcat(X, Hitters.Salary)

Row,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League__A,League__N,Division__E,Division__W,PutOuts,Assists,Errors,NewLeague__A,NewLeague__N,x1
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,0.0,1.0,0.0,1.0,632.0,43.0,10.0,0.0,1.0,475.0
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,1.0,0.0,0.0,1.0,880.0,82.0,14.0,1.0,0.0,480.0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,0.0,1.0,1.0,0.0,200.0,11.0,3.0,0.0,1.0,500.0
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,0.0,1.0,1.0,0.0,805.0,40.0,4.0,0.0,1.0,91.5
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,1.0,0.0,0.0,1.0,282.0,421.0,25.0,1.0,0.0,750.0
6,185.0,37.0,1.0,23.0,8.0,21.0,2.0,214.0,42.0,1.0,30.0,9.0,24.0,0.0,1.0,1.0,0.0,76.0,127.0,7.0,1.0,0.0,70.0
7,298.0,73.0,0.0,24.0,24.0,7.0,3.0,509.0,108.0,0.0,41.0,37.0,12.0,1.0,0.0,0.0,1.0,121.0,283.0,9.0,1.0,0.0,100.0
8,323.0,81.0,6.0,26.0,32.0,8.0,2.0,341.0,86.0,6.0,32.0,34.0,8.0,0.0,1.0,0.0,1.0,143.0,290.0,19.0,0.0,1.0,75.0
9,401.0,92.0,17.0,49.0,66.0,65.0,13.0,5206.0,1332.0,253.0,784.0,890.0,866.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1100.0


3.3 Haz una regresión OSL y predice el salario de los Hitters usando todas las características del dataset y proporciona intervalos de confianza bootstrap

In [168]:
using StatsModels
using GLM

X_vars = Symbol.(names(X))
Salary = Hitters.Salary

model_formula = @formula(Salary ~ $(join(X_vars, " + ")))

LoadError: LoadError: ArgumentError: interpolation with $ not supported in @formula.  Use @eval @formula(...) instead.
in expression starting at In[168]:7

In [48]:
using Statistics

N = nrow(Hitters)
Boot_reps = 10000
Y_hat_bootstrap = zeros(Boot_reps)

for i in 1:Boot_reps
    Y_bootstrap = rand(Boot_reps, N)
    Y_hat_bootstrap[i] = mean(Y_bootstrap)
end

println("Creamos el vector de B_hat_boots:")
B_hat_boot = rand(Boot_reps, N)

Creamos el vector de B_hat_boots:


10000×263 Matrix{Float64}:
 0.339244   0.773537   0.0808173  …  0.518323   0.53725   0.999601
 0.383738   0.192218   0.743327      0.729472   0.316701  0.965987
 0.689514   0.539809   0.524315      0.518689   0.566412  0.0400008
 0.530946   0.0122695  0.83262       0.163464   0.120104  0.261369
 0.48237    0.307493   0.91917       0.146891   0.346369  0.828299
 0.252274   0.372526   0.350457   …  0.180362   0.411003  0.57587
 0.333878   0.791699   0.222861      0.974897   0.789758  0.735722
 0.441965   0.0591824  0.55588       0.21989    0.519456  0.251246
 0.467006   0.509064   0.907581      0.667842   0.122263  0.560039
 0.223126   0.1578     0.371354      0.986012   0.983256  0.444088
 0.480698   0.0616754  0.0124986  …  0.670657   0.14321   0.769541
 0.392418   0.24853    0.330743      0.0932704  0.430037  0.0202539
 0.899171   0.690329   0.526703      0.431965   0.639213  0.176557
 ⋮                                ⋱  ⋮                    
 0.630512   0.906461   0.0761811     0.210

In [165]:
Lower_bound_boots = quantile(B_hat_boot, 0.025)
Upper_bound_boots = quantile(B_hat_boot, 0.975)
println("Calculamos")
println(Lower_bound_boots)
println(Upper_bound_boots)

LoadError: MethodError: no method matching quantile!(::Matrix{Float64}, ::Float64; sorted::Bool, alpha::Float64, beta::Float64)

[0mClosest candidates are:
[0m  quantile!([91m::AbstractVector[39m, ::Real; sorted, alpha, beta)
[0m[90m   @[39m [32mStatistics[39m [90mD:\Riot Games\Julia-1.10.5\share\julia\stdlib\v1.10\Statistics\src\[39m[90m[4mStatistics.jl:976[24m[39m
[0m  quantile!(::AbstractArray, [91m::AbstractVector[39m, [91m::AbstractArray[39m; sorted, alpha, beta)
[0m[90m   @[39m [32mStatistics[39m [90mD:\Riot Games\Julia-1.10.5\share\julia\stdlib\v1.10\Statistics\src\[39m[90m[4mStatistics.jl:943[24m[39m
[0m  quantile!(::AbstractArray, [91m::AbstractArray[39m, [91m::Union{Tuple{Vararg{Real}}, AbstractArray}[39m; sorted, alpha, beta)
[0m[90m   @[39m [32mStatistics[39m [90mD:\Riot Games\Julia-1.10.5\share\julia\stdlib\v1.10\Statistics\src\[39m[90m[4mStatistics.jl:972[24m[39m
[0m  ...


In [55]:
println("Calculamos la media de la raíz del error fuera del modelo:")
MSE = sqrt(mean((Y[test] .- predictions).^2))

Calculamos la media de la raíz del error fuera del modelo:


214.76305971641648

3.4 Haz una regresión del árbol de decisiones usando todas las características del set.

In [163]:
DecisionTreeRegressor = (@load DecisionTreeRegressor pkg=DecisionTree verbosity=0)
tree_model = DecisionTreeRegressor()
tree_machine = machine(tree_model, X[train, :], Y[train])
fit!(tree_machine);

[ Info: Training machine(DecisionTreeRegressor(max_depth = -1, …), …).


In [171]:
tree_model_prune = DecisionTreeRegressor(post_prune = true, merge_purity_threshold = 0.09)
tree_machine_prune = machine(tree_model_prune, X[train, :], Y[train])
fit!(tree_machine_prune);

[ Info: Training machine(DecisionTreeRegressor(max_depth = -1, …), …).


In [172]:
thresholds = exp.(collect(-10:0.01:0))
rmses = []

for threshold in thresholds
        tree_model_prune.merge_purity_threshold = threshold
        evaluation = evaluate!(
                tree_machine_prune,
                resampling = CV(nfolds = 3, shuffle = true, rng = 123),
                measure = rmse
        )
        rmses = [rmses; evaluation.measurement]
end

Evaluating over 3 folds: 100%[=========================] Time: 0:00:04
